In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import gc
tqdm.pandas()
gc.collect()

0

# LOAD

In [2]:
%%time
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather').dropna(axis=0)
df = df.loc[df.dt>=12,:] #取最近一年即可
print(df.shape,df.isnull().sum().sum())
df.head()

(18552970, 53) 0
Wall time: 1min 11s


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


#  以下根據chid做groupby操作,並且因應各種不同類型的欄位作客製化統計運算(沿時間維度取mean之類的)

In [3]:
%%time
def create_groupby_chid_df(df):
    
    # 1 groupby('chid')根據時間維度取最後一筆
    final_col = ['age','primary_card','trdtp','educd','gender_code','masts','poscd','naty','cuorg'] 
    var1 = df.groupby('chid')[final_col].agg(lambda x:x.values[-1]) # 最後一筆 
    
    # 2 groupby('chid')根據時間維度算平均百分比
    mean_col = df.columns[df.columns.str.contains('pct')]
    var2 = df.groupby('chid')[mean_col].agg('mean')
    
    # 3 groupby('chid')根據時間維度算norm_cnt(月平均消費次數(cnt取mean),再接一個normalize操作)
    card_cnt_col = df.columns[df.columns.str.contains('card')&df.columns.str.contains('cnt')]
    var3 = df.groupby('chid')[card_cnt_col].agg('mean')
    var3.loc[:,:] = normalize(var3)
    
    # 4 groupby('chid')根據時間維度算norm_cnt(月平均消費次數(cnt取mean),再接一個normalize操作)
    domestic_overseas_col = df.columns[(df.columns.str.contains('domestic')|df.columns.str.contains('overseas'))&(df.columns.str.contains('cnt'))]
    var4 = df.groupby('chid')[domestic_overseas_col].agg('mean')
    var4.loc[:,:] = normalize(var4)

    # 5 groupby('chid')根據時間維度算平均值(slam(正卡信用額度平均值),txn_amt(消費金額平均值),txn_cnt(消費次數平均值))
    other_mean_col = ['slam','txn_amt','txn_cnt']
    var5 = df.groupby('chid')[other_mean_col].agg('mean')

    # 1 2 3 4 5 concat 起來
    df_groupby_chid = pd.concat([var1,var2,var3,var4,var5],axis=1).reset_index()
    
    print(set(df.columns) - set(df_groupby_chid.columns))

    return df_groupby_chid

print(df.shape)
df_groupby_chid = create_groupby_chid_df(df)
df_groupby_chid

(18552970, 53)
{'dt', 'shop_tag'}
Wall time: 45.3 s


,chid,age,primary_card,trdtp,educd,gender_code,masts,poscd,naty,cuorg,...,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,slam,txn_amt,txn_cnt
0,10000000.0,4.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,30.0,...,0.0,0.0,1.000000,0.433816,0.901002,0.0,0.000000,79557.742574,9701.914585,1.600000
1,10000001.0,5.0,1.0,11.0,4.0,0.0,2.0,3.0,1.0,30.0,...,0.0,0.0,0.000000,0.969334,0.245747,0.0,0.000000,62730.806297,7977.560539,1.816327
2,10000002.0,3.0,1.0,13.0,4.0,0.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.894427,0.447214,0.0,0.000000,79291.240239,6647.343469,1.000000
3,10000003.0,6.0,1.0,15.0,6.0,0.0,2.0,99.0,1.0,30.0,...,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.000000,73926.209680,8288.806894,1.500000
4,10000004.0,2.0,1.0,15.0,5.0,1.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.666667,0.666667,0.0,0.333333,48209.277501,7076.586473,1.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474200,10499995.0,4.0,1.0,5.0,3.0,1.0,1.0,99.0,1.0,30.0,...,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,87397.101503,3716.334885,1.000000
474201,10499996.0,3.0,1.0,11.0,3.0,1.0,2.0,2.0,1.0,30.0,...,0.0,0.0,0.000000,0.972387,0.233373,0.0,0.000000,55463.131267,11491.514477,1.291667
474202,10499997.0,3.0,1.0,8.0,6.0,0.0,1.0,2.0,1.0,30.0,...,0.0,0.0,0.000000,0.976187,0.216930,0.0,0.000000,72218.713500,6568.010877,1.692308
474203,10499998.0,4.0,1.0,13.0,6.0,1.0,2.0,99.0,1.0,30.0,...,0.0,0.0,0.000000,0.768980,0.000000,0.0,0.639273,62417.960432,13384.207376,3.304348


# 做資料預處理(one_hot和標準化轉換),把資料欄位都變成數值型態特徵

In [4]:
%%time
from sklearn.preprocessing import scale
import warnings 
warnings.filterwarnings('ignore')

def preprocess_for_knn(df_groupby_chid):
    # 對類別欄位做one hot encoding
    categorical_df = pd.DataFrame()
    categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
    for c_name in categorical_features:
        df_groupby_chid[c_name] = pd.to_numeric(df_groupby_chid[c_name])
    for c_name in categorical_features:
        one_hot = pd.get_dummies(df_groupby_chid[c_name])
        one_hot.columns = [ c_name + '_' + str(i) for i in one_hot.columns]
        if len(categorical_df)==0:
            categorical_df = one_hot
        else:
            categorical_df = pd.concat([categorical_df,one_hot],axis=1)
    df_groupby_chid = df_groupby_chid.drop(categorical_features,axis=1) # 刪掉原始類別欄位
    df_groupby_chid = pd.concat([df_groupby_chid,categorical_df],axis=1) # 加入one hot版本類別欄位
    
    # 做標準化轉換
    scale_col = set(df_groupby_chid.columns) - set(['chid'])
    scale_col = list(scale_col)
    df_groupby_chid[scale_col] = scale(df_groupby_chid[scale_col])
    
    return df_groupby_chid

df_groupby_chid_preprocessed = preprocess_for_knn(df_groupby_chid)
df_groupby_chid_preprocessed


Wall time: 4.13 s


,chid,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,...,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,age_9.0,primary_card_0.0,primary_card_1.0
0,10000000.0,-0.495512,0.769734,-0.09469,-0.433920,-0.434485,-0.435813,-0.14697,-0.638494,-0.194871,...,-0.421456,-0.628111,1.667126,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
1,10000001.0,0.366034,-0.192477,-0.09469,-0.433920,0.429507,1.626150,-0.14697,-0.638494,-0.194871,...,-0.421456,-0.628111,-0.599835,2.093679,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
2,10000002.0,-0.000972,0.217411,-0.09469,-0.433920,-0.434485,-0.435813,-0.14697,1.729064,-0.194871,...,-0.421456,1.592075,-0.599835,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
3,10000003.0,1.122984,-1.037869,-0.09469,-0.433920,-0.434485,-0.435813,-0.14697,-0.638494,-0.194871,...,-0.421456,-0.628111,-0.599835,-0.477628,3.160848,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
4,10000004.0,-0.900136,0.635837,-0.09469,0.799683,-0.434485,-0.435813,-0.14697,1.729064,-0.194871,...,2.372726,-0.628111,-0.599835,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474200,10499995.0,-2.248884,-1.037869,-0.09469,7.496384,-0.434485,-0.435813,-0.14697,-0.638494,-0.194871,...,-0.421456,-0.628111,1.667126,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
474201,10499996.0,0.561006,-0.410229,-0.09469,-0.433920,-0.434485,2.426402,-0.14697,-0.638494,-0.194871,...,-0.421456,1.592075,-0.599835,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
474202,10499997.0,0.414892,-0.247043,-0.09469,-0.433920,2.655704,-0.435813,-0.14697,-0.638494,-0.194871,...,-0.421456,1.592075,-0.599835,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368
474203,10499998.0,0.756477,-1.037869,-0.09469,0.428070,-0.434485,-0.435813,-0.14697,-0.638494,-0.194871,...,-0.421456,-0.628111,1.667126,-0.477628,-0.316371,-0.136196,-0.042276,-0.007405,-0.208368,0.208368


# 特徵維度太多先用pca降維

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=14)
X = df_groupby_chid_preprocessed.drop('chid',axis=1).values
X_pca = pca.fit_transform(X)
X_pca.shape

(474205, 14)

# 建立knn模型

In [23]:
%%time
from sklearn.neighbors import NearestNeighbors

k = 4
nbrs = NearestNeighbors(
    n_neighbors=k, 
    algorithm='ball_tree',
    leaf_size=300,
    n_jobs=-1,
    p=2).fit(X_pca)
nbrs

Wall time: 2.23 s


NearestNeighbors(algorithm='ball_tree', leaf_size=300, n_jobs=-1, n_neighbors=4)

# 試用knn模型

In [24]:
distances, indices = nbrs.kneighbors(X_pca[[0]]) # 將該chid的"特徵"丟入"knn"
source_chid = df_groupby_chid_preprocessed.iloc[0]['chid']
target_chids = df_groupby_chid_preprocessed.loc[indices[0][-(k-1):]]['chid'].values.tolist()
print(source_chid,'==>',target_chids)

10000000.0 ==> [10141830.0, 10160475.0, 10030875.0]


In [27]:
source_chid_shop_tag = df.loc[df.chid == source_chid,'shop_tag'].value_counts()[:3].index.values.tolist()
target_chid_shop_tag = df.loc[df.chid.isin(target_chids),'shop_tag'].value_counts()[:3].index.values.tolist()
print(source_chid_shop_tag,target_chid_shop_tag)
print('交集百分比:',len(set(source_chid_shop_tag)&set(target_chid_shop_tag))/3)

['10', '45', 'other'] ['other', '10', '33']
交集百分比: 0.6666666666666666


# 以上實驗驗證了相似的chid有相似的shop_tag
# 因此未來在預測上如果當下chid無法集滿三個shop_tag,可以搜索相似chid的shop_tag

In [29]:
import joblib

# 保存knn模型
joblib.dump(nbrs,'../model/nbrs.pkl')
# 保存X_pca
joblib.dump(X_pca,'../model/X_pca_for_knn.pkl')
# 保存df_groupby_chid_preprocessed
df_groupby_chid_preprocessed.to_feather('../data/df_groupby_chid_preprocessed.feather')